In [2]:
import numpy as np
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.tokenize import word_tokenize
import fasttext
import fasttext.util
from numpy.linalg import norm
from tqdm import tqdm
from scipy import stats
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F


/home2/arihanth.srikar/anaconda3/envs/pytorch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
with open('data/train_data.pkl', 'rb') as f:
    train_data = pickle.load(f)

In [4]:
X = train_data['x']
y = train_data['y']

In [ ]:
ft = fasttext.load_model('/scratch/arihanth.srikar/models/cc.en.300.bin')
ft.get_dimension()

In [ ]:
# fasttext.util.reduce_model(ft, 100)
# ft.get_dimension()

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
final_embeddings = []
for (eng_sent, esp_sent) in X:
    eng_vect = torch.cat([torch.tensor([ft.get_sentence_vector(word)]) for word in eng_sent], dim=0).unsqueeze(dim=1)
    esp_vect = torch.cat([torch.tensor([ft.get_sentence_vector(word)]) for word in esp_sent], dim=0).unsqueeze(dim=1)
    final_embeddings.append((eng_vect, esp_vect))

NameError: name 'ft' is not defined

In [7]:
from flair.embeddings import WordEmbeddings
from flair.data import Sentence

In [8]:
glove_embedder = WordEmbeddings('glove')

In [9]:
final_embeddings = []
for (e1, e2) in X:
    v1 = Sentence(e1)
    v2 = Sentence(e2)
    v1 = [token.embedding.cpu() for token in glove_embedder.embed(v1)[0]]
    v2 = [token.embedding.cpu() for token in glove_embedder.embed(v2)[0]]
    final_embeddings.append((v1, v2))

In [24]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, sentences, scores):
        self.labels = scores
        self.sentences = sentences

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        return self.labels[idx]

    def get_batch_embeddings(self, idx):
        e1, e2 = self.sentences[idx]
        return torch.vstack(e1), torch.vstack(e2)

    def __getitem__(self, idx):

        embeddings = self.get_batch_embeddings(idx)
        label = self.get_batch_labels(idx)

        return embeddings, torch.tensor(label)

In [25]:
full_dataset = Dataset(final_embeddings, y)

In [26]:
(e1,e2), lbl = next(iter(full_dataset))

tensor([[-0.2709,  0.0440, -0.0203,  ..., -0.4923,  0.6369,  0.2364],
        [ 0.5937,  0.4482,  0.5932,  ..., -0.5465,  0.1516, -0.3075],
        [ 0.2616,  0.4472, -0.0968,  ..., -0.4503,  0.4952, -0.2030],
        ...,
        [ 0.0523,  0.4112, -0.5290,  ..., -0.9391,  0.9866, -0.0713],
        [-0.1440,  0.3255,  0.1426,  ...,  0.2540,  1.1078, -0.0731],
        [ 0.6848, -0.3764, -0.0787,  ...,  0.4923,  0.1823, -0.0202]])

In [27]:
class LSTM(nn.Module):

    def __init__(self, embedding_dim=300, hidden_dim=256, output_size=300):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = 1

        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=self.num_layers, bidirectional=True)

    def forward(self, v1):
        out, (h, c) = self.lstm(v1.view(v1.shape[0], 1, -1))
        out = torch.mean(h.view(h.shape[0], -1), dim=0)
        
        return out

In [28]:
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
trainloader, testloader = torch.utils.data.random_split(full_dataset, [train_size, test_size])

In [32]:
model = LSTM(100, 256, 300).to(device)
loss_function = F.mse_loss
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001)

In [33]:
# model = torch.load('/scratch/arihanth.srikar/model_saves/model.pt')

In [34]:
for epoch in range(15):

    model.train()
    pred, lbls = [], []
    running_loss = 0.0
    for (e1, e2), lbl in tqdm(trainloader):
        model.zero_grad()

        e1 = e1.to(device)
        e2 = e2.to(device)
        lbl = lbl.to(device)

        # Run our forward pass.
        v1 = model(e1)
        v2 = model(e2)

        cos = nn.CosineSimilarity(dim=0)
        output = ((cos(v1, v2)+1)/2)*5

        # Compute the loss, gradients, and update the parameters by
        loss = loss_function(output.float(), lbl.float())
        running_loss += loss.item()
        loss.backward()
        optimizer.step()

        pred.append(output.float().item())
        lbls.append(lbl.float().item())

    print(f'{epoch}-Train')
    print('Pearson:', stats.pearsonr(pred, lbls)[0]*100)
    print('Loss:', running_loss/len(trainloader))

    model.eval()
    pred, lbls = [], []
    with torch.no_grad():
        running_loss = 0.0
        for (e1, e2), lbl in tqdm(testloader):
            e1 = e1.to(device)
            e2 = e2.to(device)
            lbl = lbl.to(device)

            v1 = model(e1)
            v2 = model(e2)

            cos = nn.CosineSimilarity(dim=0)
            output = ((cos(v1, v2)+1)/2)*5

            loss = loss_function(output.float(), lbl.float())
            running_loss += loss.item()

            pred.append(output.float().item())
            lbls.append(lbl.float().item())

        print(f'{epoch}-Test')
        print('Pearson:', stats.pearsonr(pred, lbls)[0]*100)
        print('Loss', running_loss/len(testloader))

100%|██████████| 3060/3060 [00:12<00:00, 241.72it/s]


0-Train
Pearson: 57.3004237612856
Loss: 0.8799223235978819


100%|██████████| 765/765 [00:01<00:00, 744.67it/s]


0-Test
Pearson: 64.72333552303313
Loss 0.7236596801012652


100%|██████████| 3060/3060 [00:12<00:00, 240.91it/s]


...

13-Test
Pearson: 74.43574095333216
Loss 0.4903849377704773


100%|██████████| 3060/3060 [00:12<00:00, 240.47it/s]


14-Train
Pearson: 96.31879091073412
Loss: 0.07870243201078223


100%|██████████| 765/765 [00:00<00:00, 770.77it/s]

14-Test
Pearson: 75.16635626470858
Loss 0.47466464452382495


In [ ]:
torch.save(model, '/scratch/arihanth.srikar/model_saves/monolingual.pt')

In [36]:
pred, lbls = [], []
for (e1, e2), lbl in full_dataset:
    
    v1 = torch.mean(e1.view(e1.shape[0], -1), dim=0)
    v2 = torch.mean(e2.view(e2.shape[0], -1), dim=0)

    cos = nn.CosineSimilarity(dim=0)
    output = cos(v1, v2)*5

    pred.append(output.float().item())
    lbls.append(lbl.float().item())

print('Pearson:', stats.pearsonr(pred, lbls)[0]*100)

Pearson: 60.73496895047812
